# 1. Business understanding

Assessing payment capacity is one of the most important issues when financial institutions need to assign credit limits. Although it might seem trivial, in some cases the information is no available, and due to the informality of some latinamercian economies (Colombia for this study case), it is important to build statistical models that can estimate the income of the customers. 

## What data do we have?

We are going to be using the data from the National Administrative Department of Statistics of Colombia [DANE] (https://www.dane.gov.co/). The database is the result of a survey conducted to more than 25k households in three major cities in Colombia for 2018. 

All the data and metadata can be found in [this link] (http://microdatos.dane.gov.co/index.php/catalog/626/). The data has 331 variables including spending behaviours and financial burden of the households.

## What question do we want to answer?

1. Can the income be modeled after the spending patterns of the household?
2. Can the income be modeled after the financial burden of the household?
3. Is there a possible way financial institutions can include this informetion in theor models?

# 2. Data Understanding

##  Environment and data

First af all, lets load all needed packages and teh data we will be working with.

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('./IEFIC_2018.csv', sep=';')
print(df.info())
df.head()

Reading the database documentation it can be found that there are some household that do not report the total income, and that have one entry for every member of the house (including children). For this reason we are going to keep only the household head and only the households that report total income.

In [ ]:
df=df[df['INGRESO_COMPLETO']==1]
df=df[df['P6050']==1]
df=df.replace(r'^\s*$', np.nan, regex=True)
df=df.replace(98, np.nan, regex=True)
df=df.replace('98', np.nan, regex=True)
df=df.replace(99, np.nan, regex=True)
df=df.replace('99', np.nan, regex=True)
df=df[df['INGTOTOB'].notna()]
df=df.reset_index(drop=True)
df.head()

After reviewing the documentation the following variables were selected. There are basicaly 3 types of variables. Spenditure, debts and investments.

In [ ]:
keep=['P10','INGTOTOB','P2439','P2461','P2168','P2471_4','P2477','P2478_1','P2478_2','P2478_3','P2478_4','P2478_5','P2478_6','P2478_7','P2478_8','P2478_9','P2478_10','P2478_11','P2478_12','P2481_1','P2481_2','P2481_3','P2481_4','P2481_5','P2481_6','P2481_7','P2481_8','P2481_9','P2481_10','P2481_11','P2481_12','P2481_13','P2481_14','P2481_15','P2481_16','P2481_17','P2481_18','P2982','P2983','P2985','P2487','P2502','P2503','P2504','P342','P2540','P2542_3','P2542_4','P2545','P2548','P2560_3','P2560_4','P2602','P2623_3','P2623_4','P2633','P2637_3','P2637_4','P2692','P2772_3','P2772_4','P2695','P2736_3','P2736_4','P2734','P2696_3','P2696_4','P2771','P2693_3','P2693_4','P2819','P2869','P622','P1136','P1239','P1421','P2584','P2962']

df=df[keep]

names=['edu_level','income','house_owner','house_value','mortage','mortage_balance','spent_edu','spent_food','spent_clothes','spent_water','spent_energy','spent_gas','spent_cell','spent_housekeep','spent_leisure','spent_health','spent_internet','spent_transport','spent_pension','extra_house','extra_home','extra_jewelry','extra_art','extra_rent','extra_vacations','extra_retirement','extra_emergency','extra_future','extra_edu','extra_debts','extra_health','extra_children','extra_wedding','extra_invest','extra_heritage','extra_remodeling','extra_savings','small_business','small_business_value','real_estate','real_estate_value','vehicles','vehicles_value','machinery','machinery_value','credit_cards','credit_cards_payment','credit_cards_balance','credit_cards_term','pawnshop','pawnshop_payment','pawnshop_balance','loans','loans_payment','loans_balance','shark','shark_payment','shark_balance','shop','shop_payment','shop_balance','union','union_payment','union_balance','edu_loan','edu_loan_payment','edu_loan_balance','friend_loan','friend_loan_payment','friend_loan_balance','stocks','stocks_value','funds','funds_value','tdc','tdc_value','savs_acc','savs_acc_value']

df.columns=names

print(df.shape)
df.head()

In [ ]:
sns.heatmap(df[['income','spent_edu','spent_food','spent_clothes','spent_water','spent_energy','spent_gas','spent_cell','spent_housekeep','spent_leisure','spent_health','spent_internet','spent_transport','spent_pension']].isnull(), cbar=False)

### Lets do some EDA over Listings

Lets beging diferentiationg numeric and categorical variables

In [ ]:
df_lis.dtypes

It seems we have some issues with the two variables 'price' and 'security_deposit'. We have to change the type to float but doing some replacements first.

In [ ]:
df_lis['price'] = df_lis['price'].str.replace('$', '')
df_lis['price'] = df_lis['price'].str.replace(',', '')
df_lis['price']=df_lis['price'].astype('float')

df_lis['host_acceptance_rate'] = df_lis['host_acceptance_rate'].str.replace('%', '')
df_lis['host_acceptance_rate']=df_lis['host_acceptance_rate'].astype('float')

In [ ]:
# Here we split numeric and categorical variables in two datasets
l_num_vars = df_lis[df_lis.select_dtypes(include=['float', 'int']).columns]
l_cat_vars = df_lis[df_lis.select_dtypes(include=['object']).columns]

In [ ]:
#Now lets plot some correlations
sns.heatmap(l_num_vars.corr())
plt.show()

There seem to be some obvoius correlations:

1. The scores have the biggest correlation between them.
2. Between the number of bedrooms and the number of bed.

But there are some that are not that simple:

1. It seems that the socre that are more correlate with price are location and cleanliness
2. Latitude seems to be more correlated with price that longitude

In [ ]:
corr_lis=l_num_vars.corr().stack().reset_index()
corr_lis.columns = ['var_1','var_2','corr']
price_corr=corr_lis[corr_lis['var_1']=='price'].sort_values(by=['corr'])  
price_corr=price_corr[price_corr['var_2']!='price']

x = price_corr['var_2']
y = price_corr['corr']

mask1 = y < 0
mask2 = y >= 0

plt.xticks(rotation = 90)
plt.bar(x[mask1], y[mask1], color = 'bisque')
plt.bar(x[mask2], y[mask2], color = 'turquoise')
plt.show()

 Checking correlations only with 'price' it is clear that the number of bedrooms/beds, therefore the size of the listing, is strongly correlated with the price. Location (latitude and longitude) have a some correlation with price.

## Reviews!

This dataset has unique id for each reviewer and detailed comments



In [ ]:
df_rev.shape

In [ ]:
df_rev.head()

In [ ]:
df_rev=df_rev.drop(['id','date','reviewer_name'],axis=1)

In [ ]:
df_rev.groupby(['listing_id']).count().sort_values(by=['comments'],ascending=False).head()

Some listing have up to 402 comments!

In [ ]:
df_rev.groupby(['reviewer_id']).count().sort_values(by=['comments'],ascending=False).head()

How much traveling does it takes to visit 37 different listing!

This dataset is pretty straight forward. In the following section we are going to try and use some sort of prebuilt sentiment analisys tool so we can clasify the comments and try to predict the price :)

# 3. Data Preparation

In this section we are going to get our databases ready for modeling, doing some data cleaning, transformation and imputation

## Reviews!

The following list shows the transformation to be made in each of the columns in this dataset

**price:** apply logarithm and Normalization 

1. **host_since:** Calculate the monts this host hase been related to Airbnb and Normalization
2 **host_response_time:** Dummify
3. **host_response_rate:** Normalization
4. **host_acceptance_rate:** Normalization
5. **host_is_superhost:** Dummify
6. **host_total_listings_count:** Normalization
7. **latitude:** Normalization
8. **longitude:** Normalization
9. **property_type:** Dummify
10. **room_type:** Dummify
11. **accommodates:** Normalization
12. **bathrooms:** Normalization
13. **bedrooms:** Normalization
14.**beds:** Normalization
15.**price:** Normalization
16.**guests_included:** Normalization
17. **number_of_reviews:** Normalization
18. **review_scores_rating:** Normalization
19. **review_scores_accuracy:** Normalization
20. **review_scores_cleanliness:** Normalization
21. **review_scores_checkin:** Normalization
22. **review_scores_communication:** Normalization
23. **review_scores_location:** Normalization
24. **review_scores_value:** Normalization
25. **instant_bookable:** Dummify

But first, we are going to perform some type changes an calculations in some variables

In [ ]:
#Here we are calculating the number of months between the most recent host and all the others

df_lis.host_since=pd.to_datetime(df_lis.host_since)
df_lis.host_since=(max(df_lis.host_since)-df_lis.host_since)
df_lis.host_since=(df_lis.host_since/ np.timedelta64(1, 'D')).astype(float)/30

In [ ]:
#Here we change the type of the percentage.

df_lis.host_response_rate = df_lis.host_response_rate.str.replace('%', '')
df_lis.host_response_rate = df_lis.host_response_rate.astype(float)

Before normalizing an getting the dummy variables for the categories, we are going to apply some prebuilt sentiment analyzer on the reviwes. We are going to use NLTK prebuilt sentyment analyzer.

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
def get_pos(comment):
    '''
    INPUT
    comment - string 
    OUTPUT
    Positive score of Nltk sentimen analyzer.
    '''
    sia = SentimentIntensityAnalyzer()
    pos=sia.polarity_scores(comment).get('pos')
    return pos
    

In [ ]:
#Now we are going to apply our funtion to every coment, and average the value by listing id

df_rev.comments = df_rev.comments.astype('str') 
df_rev['positive']=df_rev.comments.apply(get_pos)
df_rev_mean=df_rev.groupby('listing_id')['positive'].mean().to_frame()

## Joining Resulting Dataframes


In [ ]:
df_lis=df_lis.set_index('id')
mergedDf = df_lis.merge(df_rev_mean, left_index=True, right_index=True)

This following function is based on the one showd in the solution of some quizes in Udacitys nanodegree.

# Something missing?

We are going to handle missin values in three ways:

1. Price: this is our lable, so we are going to drop all null values.
2. Numeric variables: The numeric values are going to be filled with the mean. 
3. Categorical variables: In this case the absence of variable in the dummies captures the null values.

This is safe for our model, becaouse there woulfd be no considerable variance for the inputed variables.


In [ ]:
def clean_data(df):
    '''
    INPUT
    df - pandas dataframe 
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector
    
    This function cleans df using the following steps to produce X and y:
    1. Drop all the rows with no salaries
    2. Create X as all the columns that are not the Salary column
    3. Create y as the Salary column
    4. Drop the Salary, Respondent, and the ExpectedSalary columns from X
    5. For each numeric variable in X, fill the column with the mean value of the column.
    5. For each numeric variable in X, perform min max normalization
    6. Create dummy columns for all the categorical variables in X, drop the original columns
    '''
    # Drop rows with missing price values
    df = df.dropna(subset=['price'], axis=0)
    y = df['price']
    y = np.log(y)
    y = (y-min(y))/(max(y)-min(y))


    #Drop price columns
    df = df.drop(['price'], axis=1)
    
    # Fill numeric columns with the mean
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)
        df[col]=(df[col]-min(df[col]))/(max(df[col])-min(df[col]))
        
    # Dummy the categorical variables
    cat_vars = df.select_dtypes(include=['object']).copy().columns
    for var in  cat_vars:
        # for each cat add dummy var, drop original column
        df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)
    
    X = df
    return X, y

In [ ]:
mergedDf.shape

In [ ]:
X, y = clean_data(mergedDf)  

In [ ]:
X.head()

In [ ]:
y.head()

Now we have the datasets ready to model

# 4. Modeling

In this section we are going to use a simple random forest to predict the price of the listings. We are going be doing the following steps:

1. Split data
2. Instantiate model and fit
4. Evaluate

## Split data

The data will bi splitted 80/20 for train and test



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [ ]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

## Instantiate and fit model

We are going to use a simple Random forest with 1000 estimators.



In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 321)
rf.fit(X_train, y_train)

## Evaluate Model

Since we did some transformations to the lable (price) for the error meassure to make sense we have to reverse this transormations to our predictions and test labels.



In [ ]:
pred = rf.predict(X_test)

In [ ]:
def deform(t_price):
    '''
    INPUT
    t_price - price with log and min/max transformation 
    OUTPUT
    price in USD.
    '''
    min_y=min(np.log(mergedDf.price))
    max_y=max(np.log(mergedDf.price))

    t_price= t_price*(max_y-min_y) + min_y
    t_price= np.exp(t_price)


    return t_price
    

In [ ]:
pred_usd= deform(pred)
y_test_usd= deform(y_test)

In [ ]:
errors = abs(pred_usd - y_test_usd)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'USD.')

In [ ]:
ax = plt.gca()
ax.scatter(y_test_usd, pred_usd, color="turquoise")
plt.xlabel("Real")
plt.ylabel("Prediction");

In [ ]:
importances = list(rf.feature_importances_)
labels = X.columns.tolist()
df_imp=pd.DataFrame({'variable': labels,'imp': importances})
df_most=df_imp.sort_values(by=['imp'],ascending=False).head(10)

#Plot with importances
fig, ax = plt.subplots()
ax.bar(df_most.variable, df_most.imp, 0.35, color='bisque')
plt.xticks(rotation=90)
plt.show()



# 5. Results, Evaluation and Insights

Regarding our initial questions we have the following:

1. Is there is a strong correlation between the size of the listing and its price?

If we consider the correlation between the number of bathrooms and how many people a listing can accommodate with size, the short answer is yes. 

2. Is Location the most important variable for demand and pricing?

Location (captured by lat and lon) is one of the most important variables to consider when we are taliing about he price of the listing. It is quite obvious locaiton will be important, but its kind of interseting finding that latitude is more important than longitud, meaning that its more relevant deciding wether to invest in real state north/south than east/west.

3. Past reviews impact future listings of the place?

The actual reviw score did not show up as one of the main variables, althoug the variable 'positive' (wich captures the sentiment of the comments) has some impact on the pricing.


## Conclusion

Wraping up, we found that there is a strong correlation between the location of a listing and its price (kind of obvious), but th real insight here is that the average sentimen of the comments is way more important than the socres given by the guests.